## Example 11 - Multiple vessels - from database

In [ ]:
import opentnsim
print('This notebook has been tested with OpenTNSim version {}'.format(opentnsim.__version__))

In [ ]:
# package(s) related to time, space and id
import datetime, time
import platform
import random
import os
import pathlib

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy

# spatial libraries 
import pyproj
import shapely.geometry
from simplekml import Kml, Style

# package(s) for data handling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# OpenTNSIM
import opentnsim.core as core
import opentnsim.graph_module as graph_module
import opentnsim.plot as plot
import opentnsim.model as model

# Used for mathematical functions
import math             

# Used for making the graph to visualize our problem
import networkx as nx  

# Graph location
location_graph = (os.path.relpath(pathlib.Path.cwd(), "notebooks"))
name_graph = "notebooks/Shape-Files/Rotterdam-Antwerpen-corridor/edges_2.shp"

# Vessel database
location_vessel_database = "notebooks/Vessels/richtlijnen-vaarwegen-2017.csv"

### Create graph

The cel below visualizes the problem. In graph theory the red dots are called *edges* and the lines are called *vertices*. Vessels (or any other object) move from edge 1 to edge 3 and from edge 4 to edge 2. The added complexity is that vertice 5-6 only allows traffic in one direction at a time. Vessels can travel simultanously in one direction.

**Important**: 

If you use windows and get the following error "ImportError: read_shp requires OGR: http://www.gdal.org/", you probably have [this issue](https://github.com/conda-forge/gdal-feedstock/issues/219). Solving it is possible by running the following commands in your terminal (as explained [here](https://gis.stackexchange.com/questions/294231/installing-gdal-with-anaconda)):

```bash
#Create a new virtual environment
conda create -n testgdal -c conda-forge gdal vs2015_runtime=14

#Activate virtual environment
activate testgdal

#Open Jupyter notebook
jupyer notebook
```

In [ ]:
graph = graph_module.Graph()
graph.from_shape(location_graph, name_graph)

In [ ]:
graph.create_graph_new_projection()
graph.plot()

### Create vessel database

In [ ]:
vessel_db = pd.read_csv(os.path.join(location_graph, location_vessel_database))
vessel_db.head()

In [ ]:
# Vessel type
Vessel = type('Vessel', 
              (core.Identifiable, core.Movable, core.HasContainer,
               core.VesselProperties, core.HasResource, core.Routeable), 
              {})

In [ ]:
generator = model.VesselGenerator(Vessel, vessel_db)

### Run simulation

In [ ]:
# Start simpy environment
simulation_start = datetime.datetime.now()
env = simpy.Environment(initial_time = time.mktime(simulation_start.timetuple()))

# Add graph to environment
graph.add_resources(list(graph.graph.edges), np.ones(len(list(graph.graph.edges))), env)
env.FG = graph.graph

In [ ]:
def start(env, vessel):
    while True:
        
        vessel.log_entry("Start sailing", env.now, "0", vessel.geometry)
        yield from vessel.move()
        vessel.log_entry("Stop sailing", env.now, "0", vessel.geometry)

        if vessel.geometry == nx.get_node_attributes(env.FG, "geometry")[vessel.route[-1]]:
            break

In [ ]:
vessels = []

# Add 10 vessels to the simulation
for i in range(10):
    random_1 = random.choice(list(graph.graph))
    random_2 = random.choice(list(graph.graph))
    path = nx.dijkstra_path(graph.graph, random_1, random_2)
    
    vessel = generator.generate(env, "Vessel " + str(i))
    vessel.route = path
    vessel.geometry = nx.get_node_attributes(env.FG, "geometry")[vessel.route[0]]
    vessels.append(vessel)
    
    # Add the movements of the vessel to the simulation
    env.process(start(env, vessel))

In [ ]:
# Run simulation
env.run()

### Check results

In [ ]:
df = pd.DataFrame.from_dict(vessel.log)
df.head()

In [ ]:
vessel.log